In [1]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

# load the dataset and the tokenizers
data = load_dataset('magicsword/train-en-zh')['train']
print(data['text'][0])
tokenizer_en = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer_zh = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
print("English vocab:")
print('length: ',len(tokenizer_en.vocab))
print('special tokens: ',tokenizer_en.all_special_tokens)
print('special tokens ids: ',tokenizer_en.all_special_ids)
print("Chinese vocab:")
print('length: ',len(tokenizer_zh.vocab))
print('special tokens: ',tokenizer_zh.all_special_tokens)
print('special tokens ids: ',tokenizer_zh.all_special_ids)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Fortune looked at its list of the 1,000 largest industrial companies, plus the 50 largest companies in six non-industrial businesses.|||《财富》将目光对准了工业企业1,000强和非工业企业50强榜单。


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

English vocab:
length:  28996
special tokens:  ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
special tokens ids:  [100, 102, 0, 101, 103]
Chinese vocab:
length:  21128
special tokens:  ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
special tokens ids:  [100, 102, 0, 101, 103]


In [5]:
def map_fn(example):
  return {'English':[t.split('|||')[0] for t in example['text']],
          'Chinese':[t.split('|||')[1] for t in example['text']]}

data = data.map(map_fn,batched=True)
data = data.remove_columns(['text'])
data['English'][:3],data['Chinese'][:3]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

(['Fortune looked at its list of the 1,000 largest industrial companies, plus the 50 largest companies in six non-industrial businesses.',
  'Are you worried about the African front?',
  'Your girlfriend was white,'],
 ['《财富》将目光对准了工业企业1,000强和非工业企业50强榜单。', '你对非洲前线战局有忧虑吗？', '你女朋友是白人，'])

In [6]:
def tokenize_fn_en(example):
  return {'tokenized_English':[tokenizer_en.tokenize(en,add_special_tokens=True) for en in example['English']]}
def tokenize_fn_zh(example):
  return {'tokenized_Chinese':[tokenizer_zh.tokenize(zh,add_special_tokens=True) for zh in example['Chinese']]}

data = data.map(tokenize_fn_en,batched=True)
data = data.map(tokenize_fn_zh,batched=True)
data = data.remove_columns(['English','Chinese'])
print(data['tokenized_English'][1])
print(data['tokenized_Chinese'][1])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

['[CLS]', 'Are', 'you', 'worried', 'about', 'the', 'African', 'front', '?', '[SEP]']
['[CLS]', '你', '对', '非', '洲', '前', '线', '战', '局', '有', '忧', '虑', '吗', '？', '[SEP]']


In [8]:
def tokenize_id_en(example):
  return {'tokenized_en_id':[tokenizer_en.convert_tokens_to_ids(en) for en in example['tokenized_English']]}
def tokenize_id_zh(example):
  return {'tokenized_zh_id':[tokenizer_zh.convert_tokens_to_ids(zh) for zh in example['tokenized_Chinese']]}

data = data.map(tokenize_id_en,batched=True)
data = data.map(tokenize_id_zh,batched=True)
print(data['tokenized_en_id'][1])
print(data['tokenized_zh_id'][1])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

[101, 2372, 1128, 4472, 1164, 1103, 2170, 1524, 136, 102]
[101, 872, 2190, 7478, 3828, 1184, 5296, 2773, 2229, 3300, 2569, 5991, 1408, 8043, 102]


In [9]:
tokenizer_en.decode(data['tokenized_en_id'][1]),tokenizer_zh.decode(data['tokenized_zh_id'][1])

('[CLS] Are you worried about the African front? [SEP]',
 '[CLS] 你 对 非 洲 前 线 战 局 有 忧 虑 吗 ？ [SEP]')

In [11]:
data.to_json('/content/drive/MyDrive/MSML641/dataset.json')

Creating json from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

54702049